In [24]:
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import cv2
from mtcnn import MTCNN
from PIL import Image

In [25]:
feature_list=np.array(pickle.load(open('embedding.pkl','rb'))) ##Loading and Converting to array.
filenames=pickle.load(open('filenames.pkl','rb'))

In [26]:
model=VGGFace(model='resnet50',include_top=False,input_shape=(224,224,3),pooling='avg')

## initializing the detector:
detector=MTCNN()

sample_img=cv2.imread('samples/sample3.jpg')
results=detector.detect_faces(sample_img)

x,y,width,height=results[0]['box']

face=sample_img[y:y+height,x:x+width]

In [27]:
# Extract sample image features:
image=Image.fromarray(face)
image=image.resize((224,224))

face_array=np.asarray(image)

face_array=face_array.astype('float32')

expanded_img=np.expand_dims(face_array,axis=0)
preprocessed_img=preprocess_input(expanded_img)
result=model.predict(preprocessed_img).flatten()
print(result.shape)

(2048,)


In [28]:
## Finding Cosine Distance:
similarity=[]
for i in range(len(feature_list)):
    similarity.append(cosine_similarity(result.reshape(1,-1),feature_list[i].reshape(1,-1))[0][0])

In [29]:
len(similarity)

8541

In [30]:
index_pos=sorted(list(enumerate(similarity)),reverse=True,key=lambda x:x[1])[0][0] ## Storing the index with which the sample resembles the most.

In [31]:
temp_img=cv2.imread(filenames[index_pos])
cv2.imshow('output',temp_img)
cv2.waitKey(0)

-1